<a href="https://colab.research.google.com/github/RishitalakshmiG/Performance-comparision---IaaS-and-PaaS/blob/main/Performance_comparision_Iaas_and_Paas_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following are the parameters IaaS and PaaS are tested on.

✔ Response Time (Avg + Trend)

✔ CPU Usage

✔ Memory Usage

✔ Latency Percentiles: P50, P90, P95, P99

✔ Throughput (Requests/sec)

✔ Error Rate (%)

✔ Scalability (low / medium / high load)

✔ Cost Efficiency

✔ Performance Variance

In [ ]:
#install dependencies
!pip install requests psutil plotly numpy pandas


In [ ]:
#import libraries
import requests
import psutil
import time
import statistics
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from concurrent.futures import ThreadPoolExecutor


In [ ]:
#adding endpoints
IAAS_URL = "https://httpbin.org/get"
PAAS_URL = "https://jsonplaceholder.typicode.com/posts/1"


In [ ]:
#helper function send a single request
def send_request(url):
    try:
        cpu_before = psutil.cpu_percent()
        mem_before = psutil.virtual_memory().percent

        start = time.time()
        r = requests.get(url, timeout=5)
        r.raise_for_status()
        end = time.time()

        cpu_after = psutil.cpu_percent()
        mem_after = psutil.virtual_memory().percent

        return {
            "success": True,
            "response_time": (end - start) * 1000,
            "cpu": cpu_after,
            "memory": mem_after
        }
    except:
        return {"success": False, "response_time": None, "cpu": None, "memory": None}


In [ ]:
#benchmark function with all metrics
def benchmark(url, runs=30, threads=1):
    results = []
    start_time = time.time()

    with ThreadPoolExecutor(max_workers=threads) as executor:
        futures = [executor.submit(send_request, url) for _ in range(runs)]
        for f in futures:
            results.append(f.result())

    end_time = time.time()

    # Extract successful results
    response_times = [r["response_time"] for r in results if r["success"]]
    cpu_usages = [r["cpu"] for r in results if r["success"]]
    mem_usages = [r["memory"] for r in results if r["success"]]
    error_rate = (1 - (len(response_times) / runs)) * 100

    # Percentiles
    if len(response_times) > 0:
        p50 = np.percentile(response_times, 50)
        p90 = np.percentile(response_times, 90)
        p95 = np.percentile(response_times, 95)
        p99 = np.percentile(response_times, 99)
    else:
        p50 = p90 = p95 = p99 = None

    throughput = len(response_times) / (end_time - start_time)

    return {
        "avg_response_time": statistics.mean(response_times) if response_times else None,
        "cpu_avg": statistics.mean(cpu_usages) if cpu_usages else None,
        "mem_avg": statistics.mean(mem_usages) if mem_usages else None,
        "p50": p50,
        "p90": p90,
        "p95": p95,
        "p99": p99,
        "throughput": throughput,
        "error_rate": error_rate,
        "variance": statistics.variance(response_times) if len(response_times) > 1 else None,
        "response_times": response_times
    }


In [ ]:
#run benchmarks
print("🚀 Running IaaS tests...")
iaas_low = benchmark(IAAS_URL, runs=20, threads=1)
iaas_med = benchmark(IAAS_URL, runs=50, threads=5)
iaas_high = benchmark(IAAS_URL, runs=100, threads=20)

print("🚀 Running PaaS tests...")
paas_low = benchmark(PAAS_URL, runs=20, threads=1)
paas_med = benchmark(PAAS_URL, runs=50, threads=5)
paas_high = benchmark(PAAS_URL, runs=100, threads=20)


🚀 Running IaaS tests...
🚀 Running PaaS tests...


In [ ]:
#collecting final metrics in a table
def summarize(name, low, med, high):
    return {
        "Model": name,
        "Avg Response (ms)": low["avg_response_time"],
        "CPU Avg (%)": low["cpu_avg"],
        "Mem Avg (%)": low["mem_avg"],
        "P95 Latency (ms)": low["p95"],
        "P99 Latency (ms)": low["p99"],
        "Throughput (RPS)": low["throughput"],
        "Error Rate (%)": low["error_rate"],
        "Variance": low["variance"]
    }

df = pd.DataFrame([
    summarize("IaaS", iaas_low, iaas_med, iaas_high),
    summarize("PaaS", paas_low, paas_med, paas_high)
])

df


,Model,Avg Response (ms),CPU Avg (%),Mem Avg (%),P95 Latency (ms),P99 Latency (ms),Throughput (RPS),Error Rate (%),Variance
0,IaaS,835.287362,13.11875,9.13125,1775.331914,3116.973007,0.852755,20.0,617231.568162
1,PaaS,51.382816,49.54500,9.80000,67.024744,72.859867,19.082118,0.0,46.555904


In [ ]:
#plot graphs
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df["Model"],
    y=df["Avg Response (ms)"],
    name="Avg Response Time"
))

fig.add_trace(go.Bar(
    x=df["Model"],
    y=df["P95 Latency (ms)"],
    name="P95"
))

fig.add_trace(go.Bar(
    x=df["Model"],
    y=df["P99 Latency (ms)"],
    name="P99"
))

fig.update_layout(
    title="Performance Comparison: Response Time & Percentiles",
    barmode="group"
)

fig.show()


In [ ]:
#throughput and error rate dashboard
fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=df["Model"],
    y=df["Throughput (RPS)"],
    name="Throughput"
))

fig2.add_trace(go.Bar(
    x=df["Model"],
    y=df["Error Rate (%)"],
    name="Error Rate"
))

fig2.update_layout(
    title="Throughput vs Error Rate",
    barmode="group"
)
fig2.show()


In [ ]:
#cost effiency calculation
iaas_cost = float(input("Enter IaaS cost per hour ($): "))
paas_cost = float(input("Enter PaaS cost per hour ($): "))

df["Cost Efficiency (RPS per $)"] = [
    df.iloc[0]["Throughput (RPS)"] / iaas_cost,
    df.iloc[1]["Throughput (RPS)"] / paas_cost
]

df


Enter IaaS cost per hour ($): 50
Enter PaaS cost per hour ($): 100


,Model,Avg Response (ms),CPU Avg (%),Mem Avg (%),P95 Latency (ms),P99 Latency (ms),Throughput (RPS),Error Rate (%),Variance,Cost Efficiency (RPS per $)
0,IaaS,835.287362,13.11875,9.13125,1775.331914,3116.973007,0.852755,20.0,617231.568162,0.017055
1,PaaS,51.382816,49.54500,9.80000,67.024744,72.859867,19.082118,0.0,46.555904,0.190821


In [ ]:
#final csv download
df.to_csv("advanced_cloud_benchmark.csv", index=False)
from google.colab import files
files.download("advanced_cloud_benchmark.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#final verdict
def final_verdict(df):
    iaas = df[df["Model"] == "IaaS"].iloc[0]
    paas = df[df["Model"] == "PaaS"].iloc[0]

    reasons = []

    # --- Latency Comparison ---
    if paas["Avg Response (ms)"] < iaas["Avg Response (ms)"]:
        reasons.append("PaaS has lower average response time.")
    else:
        reasons.append("IaaS has lower average response time.")

    # --- Percentile Latency ---
    if paas["P95 Latency (ms)"] < iaas["P95 Latency (ms)"]:
        reasons.append("PaaS shows better worst-case latency (P95).")
    else:
        reasons.append("IaaS shows better worst-case latency (P95).")

    # --- Throughput ---
    if paas["Throughput (RPS)"] > iaas["Throughput (RPS)"]:
        reasons.append("PaaS supports higher throughput per second.")
    else:
        reasons.append("IaaS supports higher throughput per second.")

    # --- Error Rate ---
    if paas["Error Rate (%)"] < iaas["Error Rate (%)"]:
        reasons.append("PaaS is more reliable with fewer errors.")
    else:
        reasons.append("IaaS is more reliable with fewer errors.")

    # --- Performance Consistency (Variance) ---
    if paas["Variance"] < iaas["Variance"]:
        reasons.append("PaaS provides more consistent performance (lower variance).")
    else:
        reasons.append("IaaS provides more consistent performance (lower variance).")

    # --- Cost Efficiency ---
    if "Cost Efficiency (RPS per $)" in df.columns:
        if paas["Cost Efficiency (RPS per $)"] > iaas["Cost Efficiency (RPS per $)"]:
            reasons.append("PaaS gives better performance per dollar.")
        else:
            reasons.append("IaaS gives better performance per dollar.")

    # --- Decide Verdict ---
    score_paas = 0
    score_iaas = 0

    for r in reasons:
        if "PaaS" in r:
            score_paas += 1
        if "IaaS" in r:
            score_iaas += 1

    if score_paas > score_iaas:
        recommended = "PaaS"
    else:
        recommended = "IaaS"

    # --- Generate HUMAN-LIKE VERDICT TEXT ---
    print("======== FINAL VERDICT ========")
    print(f"Recommended Cloud Model: {recommended}")

    if recommended == "PaaS":
        print("Reason: Based on overall speed, scalability, lower latency percentiles, and reliability, "
              "PaaS is better suited for latency-sensitive or auto-scaling workloads.")
    else:
        print("Reason: IaaS provides more stable performance, better consistency, and lower variance, "
              "making it ideal for CPU-heavy or control-oriented workloads.")

    print("\nDetailed Comparison Reasons:")
    for r in reasons:
        print(f"• {r}")

# run the verdict function
final_verdict(df)


======== FINAL VERDICT ========
Recommended Cloud Model: PaaS
Reason: Based on overall speed, scalability, lower latency percentiles, and reliability, PaaS is better suited for latency-sensitive or auto-scaling workloads.

Detailed Comparison Reasons:
• PaaS has lower average response time.
• PaaS shows better worst-case latency (P95).
• PaaS supports higher throughput per second.
• PaaS is more reliable with fewer errors.
• PaaS provides more consistent performance (lower variance).
• PaaS gives better performance per dollar.
